# Evaluate Plots V2

test pair에 대한 iteration을 수행한다.

1. pickle을 통해서 모든 아이템을 다 읽는다.
2. 현재 source sequence를 읽는다.
3. scene similarity를 읽는다.
- mistral로 계산한 inclusion ratio
- 직접 계산한 entity similarity

4. action sequence similarity를 계산한다.
- agent4 필터 통과와 상관없이 플롯
- agent4 필터를 통과하면 플롯

5. activity taxnomy similarity를 계산한다.
- agent4 필터 통과와 상관없이 플롯
- agent4 필터를 통과하면 플롯


다음의 pandas colum을 구성한다. (568x13 정도 되나?)

없는 곳은 None표시를 한다.

source idx, target idx, 
augmentation_id, inclusion_ratio, entity_similarity
goal_category, goal_description
core activity_gt, core activity_inf
sequence sim, taxonomy sim
sequence bool, taxonomy bool

In [17]:
# %matplotlib qt
import sys
import os
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
import mplcursors
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
from bert_score import score
import pickle

#soft dtw
import torch
from tslearn.metrics import soft_dtw

sys.path.append(os.path.abspath('/root/project')) # add root path to sys.path
sys.path.append(os.path.abspath('/usr/local/lib/python3.10/dist-packages'))
from util import util_constants
from util import util_funcs
import f4_evaluate.evaluate_scene as evaluate_scene
import f1_init.database_init as database_init
import f1_init.agent_init as agent_init
import f1_init.constants_init as constants_init

#Computing similarity
import json
from sentence_transformers import SentenceTransformer
from bert_score import score
import numpy as np
from numpy.linalg import norm
import ast
import pandas as pd

#PATHS
PATH_CURR_FOLDER = os.path.abspath('') 
PATH_DATA_ANNOTATION = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_annotation'))
PATH_DATA_INPUT_OUTPUT = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output'))
PATH_SOURCE_TARGET_INPUT = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/input/source_target_video_list.pkl'))
TEST_SPATIAL_ANNOTATION_PATH_MANUAL = PATH_DATA_ANNOTATION + '/spatial_testset/manual'

# #PAIRSIM DATA#NOT USED
# PATH_PAIRSIM = PATH_DATA_ANNOTATION + '/spatial_pairsim_result'

# #AUGMENTED DATA PATH#NOT USED
# TEST_SPATIAL_ANNOTATION_PATH_SEMI = PATH_DATA_ANNOTATION + '/spatial_testset/semi'    
# #NOT USED
# TEST_SPATIAL_ANNOTATION_V2_PATH = PATH_DATA_ANNOTATION + '/spatial_augmentation/TESTSET_Augmented_Data_v2/'

#BASELINE RESULT PATHS for v5
# PATH_BASELINE_RAG = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-rag-0530-runall-newratio/'))
# PATH_BASELINE_NORAG = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-norag-0530-runall-newratio/'))
# PATH_BASELINE_1DIRECT = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-1direct-0530-runall-newratio/'))
# PATH_BASELINE_1GOALMEDIATION = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-1goalmediation-0530-runall-newratio/'))

#BASELINE FROM v6
PATH_BASELINE_RAG = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-rag-0602/'))
PATH_BASELINE_NORAG = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-norag-0602/'))
PATH_BASELINE_RAGGOAL = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-1goalmediation-0602/'))
PATH_BASELINE_NORAGGOAL = os.path.abspath(os.path.join(PATH_CURR_FOLDER, '..', 'data_input_output/output-1goalmediation-norag-0602/'))


In [18]:
def load_file(path):
    try:          
         with open(path, "rb") as f:
             data = pickle.load(f)
             return data
    except:
         return None
    
def check_file(path):
    try:
        with open(path, "rb") as f:
            pickle.load(f)  # try loading to ensure file is not empty or corrupted
        return True
    except (EOFError, FileNotFoundError, PermissionError, IsADirectoryError, pickle.UnpicklingError):
        return False


def normalize_to_string(value):
    '''
    used so to cope with value from a dictionary that happens to be inside a list bracket    
    '''
    # If it's a list and the first element is a string, return that
    if isinstance(value, list) and len(value) > 0 and isinstance(value[0], str):
        return value[0]
    # If it's already a string, return it
    elif isinstance(value, str):
        return value
    # Fallback for anything else
    else:
        return str(value)
    

#=====================================
# SIMILARITIES v1 SINGLE STRING SBERT MODEL
#=====================================
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    if norm(vec1) == 0 or norm(vec2) == 0:
        return 0.0
    return np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))

def safe_parse_sequence(seq):
    if seq is None:
        return []
    if isinstance(seq, list):
        return seq
    try:
        return ast.literal_eval(seq)
    except (ValueError, SyntaxError):
        print("taxonomy safe parse value error")
        return []

def safe_parse_taxonomy(tax):
    if tax is None:
        return {}
    if isinstance(tax, dict):
        return tax
    try:
        return json.loads(tax)
    except (json.JSONDecodeError, TypeError):
        return {}

def compute_similarities(entry, embed_model):
    # --- Parse sequences ---
    source_seq = safe_parse_sequence(entry.get('source_sequence'))
    target_seq = safe_parse_sequence(entry.get('target_sequence'))

    if not source_seq or not target_seq:
        seq_similarity = 0.0
    else:
        source_seq_str = ' '.join(source_seq)
        target_seq_str = ' '.join(target_seq)
        source_seq_emb = embed_model.encode(source_seq_str)
        target_seq_emb = embed_model.encode(target_seq_str)
        seq_similarity = cosine_similarity(source_seq_emb, target_seq_emb)

    # --- Parse taxonomies ---
    source_tax = safe_parse_taxonomy(entry.get('source_taxonomy'))
    target_tax = safe_parse_taxonomy(entry.get('target_taxonomy'))

    if not source_tax or not target_tax:
        tax_similarity = 0.0
    else:
        source_tax_str = ' '.join(f"{k}: {v}" for k, v in source_tax.items())
        target_tax_str = ' '.join(f"{k}: {v}" for k, v in target_tax.items())
        source_tax_emb = embed_model.encode(source_tax_str)
        target_tax_emb = embed_model.encode(target_tax_str)
        tax_similarity = cosine_similarity(source_tax_emb, target_tax_emb)

    return seq_similarity, tax_similarity

#=====================================
# SIMILARITIES v2 WEIGHTED LINEAR TAXONOMY SEMANTICS MODEL
# weight by level
# Compare only values
#=====================================
def compute_weighted_tax_similarity(entry, embed_model):
    '''
    func: calculate weighted taxonomy for values only
    input: entry: dict element of baseline results
    input: embed_model: stuff like sbert of openaiembedding
    w = [0.5,0.2,0.1,0.1,0.1]
    no source or target: return 0
    source empty for 0th level: return 0
    1 empty/impossible for both in non-0th: consider as +1.0*w[i]
    2 one empty/impossible: +0.0*w[i]
    3 +sim(val_source, val_target)*w[i]
    return 
    '''
    weighted_tax_similarity = 0
    weights = [0.5, 0.2, 0.1, 0.1, 0.1]
    source_tax = safe_parse_taxonomy(entry.get('source_taxonomy'))
    target_tax = safe_parse_taxonomy(entry.get('target_taxonomy'))

    if not source_tax or not target_tax:
        weighted_tax_similarity = 0.0
        print(f"No files: entry: {entry}")
        return weighted_tax_similarity
    else:
        source_items = list(source_tax.items())
        target_items = list(target_tax.items())
        weighted_tax_similarity = 0

        for i in range(len(source_items)):

            if len(source_items) != 5:
                print(f"items {len(source_items)} {len(target_items)}")
                weighted_tax_similarity = 0.0
                return weighted_tax_similarity
            
            source_key, source_value = source_items[i]
            target_key, target_value = target_items[i]
            source_value = normalize_to_string(source_value)
            target_value = normalize_to_string(target_value)

            # set impossible to empty
            if source_value =="impossible":
                source_value = "empty"
            if target_value == "impossible":
                target_value = "empty"


            # source is empty for most upper ZERO similarity
            if i == 0 and source_value == "empty":
                weighted_tax_similarity = 0.0
                print(f"0th level empty: entry: {entry}")
                return weighted_tax_similarity
            
            # source is there but target has value
            if source_value != "empty" and target_value == "empty":
                weighted_tax_similarity += 0.0
            elif source_value == "empty" and target_value != "empty":
                weighted_tax_similarity += 0.0
            elif source_value == "empty" and target_value == "empty":
                weighted_tax_similarity += weights[i]
            else:
                print(f"{i}th level: {source_value} {target_value}")
                source_value_emb = embed_model.encode(source_value)
                target_value_emb = embed_model.encode(target_value)
                weighted_tax_similarity += weights[i] * cosine_similarity(source_value_emb, target_value_emb)

    return weighted_tax_similarity

def compute_dtw_seq_similarity(entry, embed_model):
    '''
    compute dynamic time warping similarity between two sequences
    '''
    seq_similarity =[]
    source_seq = safe_parse_sequence(entry.get('source_sequence'))
    target_seq = safe_parse_sequence(entry.get('target_sequence'))

    if not source_seq or not target_seq:
        seq_similarity = 0.0
    else:
        # Get embeddings for each step in both sequences
        emb1 = embed_model.encode(source_seq)
        emb2 = embed_model.encode(target_seq)
        distance, path = fastdtw(emb1, emb2, dist=cosine)
        # Normalize distance to similarity (lower distance = higher similarity)
        # Optional: convert to similarity score in [0, 1] range
        max_possible_distance = len(path)  # worst case = all pairs unrelated
        seq_similarity = 1 - (distance / max_possible_distance)

    return seq_similarity


def compute_soft_dtw(entry, embed_model):
    '''
    compute dynamic time warping similarity between two sequences
    '''
    seq_similarity =[]
    source_seq = safe_parse_sequence(entry.get('source_sequence'))
    target_seq = safe_parse_sequence(entry.get('target_sequence'))  

    if not source_seq or not target_seq:
        seq_similarity = 0.0
    else:
        # Get embeddings for each step in both sequences
        emb1 = embed_model.encode(source_seq)
        emb2 = embed_model.encode(target_seq)
        distance, path = fastdtw(emb1, emb2, dist=cosine)
        # Normalize distance to similarity (lower distance = higher similarity)
        # Optional: convert to similarity score in [0, 1] range
        max_possible_distance = len(path)  # worst case = all pairs unrelated
        seq_similarity = 1 - (distance / max_possible_distance)

    return seq_similarity

#=====================================
# SIMILARITIES v3 WEIGHTED SIMILARITY & GED MODEL
# COMPLETELY DIFFERENT WEIGHT FOR insertion, deletion of nodes
# 
#=====================================


## Read Results

read a list of 568 pickle sets in 8 elements for 8 levels in augmentation



make dictionary
-fill every information
-fill every calculatable information
-save as df

In [19]:

source_list = [] # [0,0,0,0,1,1,1,1,...,283] -> [0,1,2,3,..71,0,1,2,3,4..71,...71]
target_list = [] # [augno_0: 55, aug33_0: 33,aug67_0: 28,aug100_0: 22,...] -> [55,26,22,...]
# baselines = ["rag","norag","1direct","1goalmediation"] #until v5
baselines = ["rag","norag","raggoal","noraggoal"]#from v6
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

def check_missing_index(baseline_result_path:str, baseline:str):
    '''
    func: return list of missing index for earlier agent set.
    output: dict_list of {idx:int, agent:"agent_name"} that if missing
    '''
    dict_list =[]
    # augmodes = [30, 40, 50, 60, 70, 80, 90, 100]#until v5
    augmodes = [33.3, 66.6, 100]#from v6
    length = 71 * len(augmodes) #8levels, 8files    

    # daseline different loadout 
    for i in range(length):
        prefix = f"pair{i}_" 
        path_source = baseline_result_path +"/"+ prefix + "sourceinfo.pkl"
        path_target = baseline_result_path +"/"+ prefix + "targetinfo.pkl"
        path_agent1a = baseline_result_path +"/"+ prefix + "agent1a.pkl"
        path_agent1b = baseline_result_path +"/"+ prefix + "agent1b.pkl"
        path_agent2a = baseline_result_path +"/"+ prefix + "agent2a.pkl"
        path_agent2b = baseline_result_path +"/"+ prefix + "agent2b.pkl"
        path_agent3 = baseline_result_path +"/"+ prefix + "agent3.pkl"
        path_agent4 = baseline_result_path +"/"+ prefix + "agent4.pkl"        
        if baseline == "1direct":
            if not check_file(path_source):
                dict_list.append({"idx": i, "agent": "sourceinfo"})
                continue
            if not check_file(path_target):
                dict_list.append({"idx": i, "agent": "targetinfo"})
                continue         
            if not check_file(path_agent3):
                dict_list.append({"idx": i, "agent": "agent3"})
                continue                
            # if not check_file(path_agent4):
            #     dict_list.append({"idx": i, "agent": "agent4"})
            #     continue                
           
        if baseline == "1goalmediation":
            if not check_file(path_source):
                dict_list.append({"idx": i, "agent": "sourceinfo"})
                continue
            if not check_file(path_target):
                dict_list.append({"idx": i, "agent": "targetinfo"})
                continue
            if not check_file(path_agent1a):
                dict_list.append({"idx": i, "agent": "agent1a"})
                continue               
            if not check_file(path_agent3):
                dict_list.append({"idx": i, "agent": "agent3"})
                continue               
            if not check_file(path_agent4):
                dict_list.append({"idx": i, "agent": "agent4"})
                continue                  

        if baseline == "raggoal" or baseline == "noraggoal" :
            if not check_file(path_source):
                dict_list.append({"idx": i, "agent": "sourceinfo"})
                continue
            if not check_file(path_target):
                dict_list.append({"idx": i, "agent": "targetinfo"})
                continue
            if not check_file(path_agent1a):
                dict_list.append({"idx": i, "agent": "agent1a"})
                continue               
            if not check_file(path_agent3):
                dict_list.append({"idx": i, "agent": "agent3"})
                continue               
            if not check_file(path_agent4):
                dict_list.append({"idx": i, "agent": "agent4"})
                continue   

        elif baseline == "rag" or baseline =="norag":
            if not check_file(path_source):
                dict_list.append({"idx": i, "agent": "sourceinfo"})
                continue
            if not check_file(path_target):
                dict_list.append({"idx": i, "agent": "targetinfo"})
                continue
            if not check_file(path_agent1a):
                dict_list.append({"idx": i, "agent": "agent1a"})
                continue       
            if not check_file(path_agent1b):
                dict_list.append({"idx": i, "agent": "agent1b"})
                continue             
            if not check_file(path_agent2a):
                dict_list.append({"idx": i, "agent": "agent2a"})
                continue          
            if not check_file(path_agent2b):
                dict_list.append({"idx": i, "agent": "agent2b"})
                continue        
            if not check_file(path_agent3):
                dict_list.append({"idx": i, "agent": "agent3"})
                continue                                    
            if not check_file(path_agent4):
                dict_list.append({"idx": i, "agent": "agent4"})
                continue               

    return dict_list                                  

def make_singlebaseline_list(baseline_result_path:str, baseline:str):
    '''
    func: make single baseline list
    '''
    baseline_result_path = baseline_result_path + "/"
    baseline_results = []
    # augmodes = [30, 40, 50, 60, 70, 80, 90, 100]#until v5
    augmodes = [33.3, 66.6, 100]#from v6
    length = 71 * len(augmodes) #8levels, 8files    

    
    for i in range(length):
    # for i in range(568):
        print(i)
        dict = {
            "idx":None,"source_idx":None, "target_idx":None, "source_uid": None, "target_uid": None, "baseline": None, "augmode": None, 
            "source_sequence": None, "source_scene_graph": None, "target_scene_graph": None, 
            "inclusion_ratio":None, #"pairwise_similarity": None,
            #"sequence_similarity_sbert": None, "taxonomy_similarity_sbert":None
            }

        prefix = f"pair{i}_"
        augmode = i%len(augmodes)
        # FILL RAW DATA
        # print(i)
        sourceinfo_dict = load_file(baseline_result_path + prefix + "sourceinfo.pkl")
        targetinfo_dict = load_file(baseline_result_path + prefix + "targetinfo.pkl")
        dict["idx"] = i
        dict["source_idx"] = sourceinfo_dict['source_idx']
        dict["target_idx"] = targetinfo_dict['target_idx']
        dict["source_uid"] = sourceinfo_dict['source_uid']
        dict["target_uid"] = targetinfo_dict['target_uid']
        dict["baseline"] = baseline
        dict["augmode"] = augmode


        dict['source_sequence'] =sourceinfo_dict['source_action_sequence']
        dict['source_sequence'] = dict['source_sequence'].strip('"').split(', ') #make into list

        dict['source_scene_graph'] =sourceinfo_dict['source_scene_graph']
        dict['target_scene_graph'] =targetinfo_dict['target_scene_graph']

        
        # based on baselines 1direct, 1goalmediation ["rag","norag","raggoal","noraggoal"]#from v6
        if baseline == "1direct":
            dict['target_sequence'] =load_file(baseline_result_path + prefix + "agent3.pkl")
        elif baseline == "1goalmediation":
            dict['target_sequence'] =load_file(baseline_result_path + prefix + "agent3.pkl")
            dict['core_activity'] =load_file(baseline_result_path + prefix + "agent1a.pkl")
        elif baseline == "raggoal" or baseline == "noraggoal":
            dict['target_sequence'] =load_file(baseline_result_path + prefix + "agent3.pkl")
            dict['core_activity'] =load_file(baseline_result_path + prefix + "agent1a.pkl")            
        elif baseline == "rag" or baseline =="norag":
            dict['target_sequence'] =load_file(baseline_result_path + prefix + "agent3.pkl")
            dict['core_activity'] =load_file(baseline_result_path + prefix + "agent1a.pkl")
            dict['source_taxonomy'] =load_file(baseline_result_path + prefix + "agent1b.pkl")
            dict['common_taxonomy'] =load_file(baseline_result_path + prefix + "agent2a.pkl")
            dict['target_taxonomy'] =load_file(baseline_result_path + prefix + "agent2b.pkl")            


        booleans= load_file(baseline_result_path+prefix+"agent4.pkl")
        if(booleans == None):
            ## No values whatsover
            dict["sequence_boolean"] = False
            dict["taxonomy_boolean"] = False 
        else:
            values = [
                line.split(":")[1].strip()
                for line in booleans.strip().splitlines()
                if ":" in line
            ]
            # print(values)
            dict["taxonomy_boolean"] = True if values[0] == 'yes' else False
            dict["sequence_boolean"] = True if values[1] == 'yes' else False
            # print(f"{dict['taxonomy_boolean']} {dict['sequence_boolean']}")
        
        dict['inclusion_ratio'] = sourceinfo_dict['spatial_similarity']
        #dict['pairwise_similarity'] = compare_scene_graph(dict['source_scene_graph'], dict['target_scene_graph'])
        # SBERT for block level semantics

        if dict['target_sequence'] != None:
            
            # Preprocess sequece to make sequence a single string
            
            dict['target_sequence'] = ast.literal_eval(dict['target_sequence'])
            print(dict['target_sequence'])
            dict['target_sequence'] = [", ".join(dict['target_sequence'])] # to single item
            dict['source_sequence'] = [", ".join(dict['source_sequence'])]# to single item
            
            # print(dict['source_taxonomy'])
            # print(dict['target_taxonomy'])

            sequence_similarity_sbert, taxonomy_similarity_sbert = compute_similarities(dict, sbert_model)
            dict["sequence_similarity_sbert"]= sequence_similarity_sbert
            dict["taxonomy_similarity_sbert"]= taxonomy_similarity_sbert

            taxonomy_weighted_similarity = compute_weighted_tax_similarity(dict, sbert_model)
            dict["taxonomy_weighted_similarity"] = taxonomy_weighted_similarity
            sequence_dtw_similarity = compute_dtw_seq_similarity(dict, sbert_model)
            dict["sequence_dtw_similarity"] = sequence_dtw_similarity
            # print(dict["taxonomy_similarity_sbert"])

            # BERTSCORE for little more stepwise precision in similarity measurement
            # print(len(dict['target_sequence']))
            # print(len(dict['source_sequence']))
            # print(dict['target_sequence'])
            # print(dict['source_sequence'])# not inside bracket
            # print(dict['target_taxonomy'])
            # print(dict['source_taxonomy'])

            #takes very long
            # dict["sequence_b_P"], dict["sequence_b_R"], dict["sequence_b_F1"] = score(dict['target_sequence'], dict['source_sequence'], lang="en", rescale_with_baseline=True)

            #This part gives trouble
            # dict["taxonomy_b_P"], dict["taxonomy_b_R"], dict["taxonomy_b_F1"] =score(dict['target_taxonomy'], dict['source_taxonomy'], lang="en", rescale_with_baseline=True)
        baseline_results.append(dict)
    return baseline_results


baseline_rag_missing_dict = check_missing_index(PATH_BASELINE_RAG, baselines[0])
baseline_norag_missing_dict = check_missing_index(PATH_BASELINE_NORAG, baselines[1])
# baseline_1direct_missing_dict = check_missing_index(PATH_BASELINE_1DIRECT, baselines[2])
# baseline_1goalmediation_missing_dict = check_missing_index(PATH_BASELINE_1GOALMEDIATION, baselines[3])
baseline_raggoal_missing_dict = check_missing_index(PATH_BASELINE_RAGGOAL, baselines[2])
baseline_noraggoal_missing_dict = check_missing_index(PATH_BASELINE_NORAGGOAL, baselines[3])

print(len(baseline_rag_missing_dict))
print(baseline_rag_missing_dict)
# print(len(baseline_norag_missing_dict))
# print(baseline_norag_missing_dict)
print(len(baseline_raggoal_missing_dict))
print(baseline_raggoal_missing_dict)
print(len(baseline_noraggoal_missing_dict))
print(baseline_noraggoal_missing_dict)

baseline_rag_list = make_singlebaseline_list(PATH_BASELINE_RAG, baselines[0])
baseline_norag_list = make_singlebaseline_list(PATH_BASELINE_NORAG, baselines[1])
# baseline_1direct_list = make_singlebaseline_list(PATH_BASELINE_1DIRECT, baselines[2])
# baseline_1goalmediation_list = make_singlebaseline_list(PATH_BASELINE_1GOALMEDIATION, baselines[3])
baseline_raggoal_list = make_singlebaseline_list(PATH_BASELINE_RAGGOAL, baselines[2])
baseline_noraggoal_list = make_singlebaseline_list(PATH_BASELINE_NORAGGOAL, baselines[3])



0
[]
0
[]
0
[]
0
[]
1
['Arrange towel', 'Arrange towel', 'Arrange towel', 'Arrange towel', 'Fold towel', 'Arrange towel', 'Place folded towel on table', 'Arrange towel on table']
0th level: cloth towel
2th level: clothes horse table
3th level: towel towel
4th level: packed arranged
2
['Arrange towel', 'Arrange towel', 'Arrange towel', 'Arrange towel', 'Fold towel', 'Arrange towel']
0th level: cloth towel
1th level: arranged folding
2th level: towel towel
3
[]
0th level: steak steak
4
['pick up steak', 'rub steak', 'rub steak', 'rub steak', 'rub steak', 'rub steak', 'rub steak', 'rub steak']
items 4 4
5
['select steak', 'open refrigerator', 'take oil from refrigerator', 'rub oil on steak']
0th level: steak steak
1th level: oil oil
6
['player picks up bell pepper', 'player moves to table', 'player deseeds bell pepper with spoon over table']
0th level: corn bell pepper
1th level: threshed deseeding
2th level: hand spoon
3th level: table table
4th level: shelled deseeded
7
['open refrigera

In [7]:
print(f"{len(baseline_rag_list)} {len(baseline_rag_list[0])}")
print(f"{len(baseline_norag_list)} {len(baseline_norag_list[0])}")
print(f"{len(baseline_raggoal_list)} {len(baseline_raggoal_list[0])}")
print(f"{len(baseline_noraggoal_list)} {len(baseline_noraggoal_list[0])}")


213 22
213 19
213 19


## Perform Numerical Calculation of Results

### base results
- pairwise scene similarity
- seq2seq similarity
- seq2seq taxonomy similarity
### plot
- per scenario similarity analysis
- per augmentation, per scenario similarity analysis
- per scenario similarity vs similarity



In [20]:

def mask_nan_df(df):
    mask = df.isna().any(axis=1)
    df.loc[mask, :] = np.nan 
    
def mask_nan_df_or_condition(df, columns_to_check):
    '''
    input: columns_to_check = ["col1", "col2"]
    '''
    mask = df[columns_to_check].isna().any(axis=1)
    df.loc[mask, :] = np.nan    

def save_baseline_list(path, baseline_list):
    with open(path, 'wb') as f:
        pickle.dump(baseline_list, f)
        print(f"saved baseline list at {path}")

# turn list of dictionary into dataframe
# df_rag = pd.DataFrame(baseline_rag_list)


df_rag = pd.DataFrame(baseline_rag_list)
# mask_nan_df_or_condition(df_rag, ["target_taxonomy","target_sequence"])

df_norag = pd.DataFrame(baseline_norag_list)
# mask_nan_df_or_condition(df_norag, ["target_taxonomy","target_sequence"])

# df_1direct = pd.DataFrame(baseline_1direct_list)
# mask_nan_df_or_condition(df_1direct, ["target_sequence"])
# # print(df_1direct.head())

# df_1goalmediation = pd.DataFrame(baseline_1goalmediation_list)
# mask_nan_df_or_condition(df_1goalmediation, ["target_sequence"])
# # print(df_1goalmediation.head())

df_raggoal = pd.DataFrame(baseline_raggoal_list)
mask_nan_df_or_condition(df_raggoal, ["target_sequence"])
# print(df_1direct.head())

df_noraggoal = pd.DataFrame(baseline_noraggoal_list)
mask_nan_df_or_condition(df_noraggoal, ["target_sequence"])
# print(df_1goalmediation.head())


baseline_rag_path = 'result_v6/rag_dict.pkl'
baseline_norag_path = 'result_v6/norag_dict.pkl'
# baseline_1direct_path = 'evaluation_result_v4/1direct_dict.pkl'
# baseline_1goalmediation_list_path = 'evaluation_result_v4/1goalmediation_dict.pkl'
baseline_raggoal_path = 'result_v6/raggoal_dict.pkl'
baseline_1noraggoal_list_path = 'result_v6/noraggoal_dict.pkl'

save_baseline_list(baseline_rag_path, baseline_rag_list)
save_baseline_list(baseline_norag_path, baseline_norag_list)
# save_baseline_list(baseline_1direct_path, baseline_1direct_list)
# save_baseline_list(baseline_1goalmediation_list_path, baseline_1goalmediation_list)
save_baseline_list(baseline_raggoal_path, baseline_raggoal_list)
save_baseline_list(baseline_1noraggoal_list_path, baseline_noraggoal_list)


print(df_rag.shape[0])
print(df_norag.shape[0])
# print(df_1direct.shape[0])
# print(df_1goalmediation.shape[0])
print(df_raggoal.shape[0])
print(df_noraggoal.shape[0])

# print(df_rag['sequence_boolean'])
# print(df_norag['sequence_boolean'])

# print(df_rag['taxonomy_boolean'])
# print(df_norag['taxonomy_boolean'])




saved baseline list at result_v6/rag_dict.pkl
saved baseline list at result_v6/norag_dict.pkl
saved baseline list at result_v6/raggoal_dict.pkl
saved baseline list at result_v6/noraggoal_dict.pkl
213
213
213
213


/tmp/ipykernel_2107141/2265981955.py:10: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_2107141/2265981955.py:10: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_2107141/2265981955.py:10: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_2107141/2265981955.py:10: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, p

In [ ]:
# MANUAL-GOLD, CH-SILVER, ELSE-BRONZE

# GOLD (manual)
# filtered_df = df[df["v1"].isin(mylist)]
gold_list = [
"dcd09fa4-afe2-4a0d-9703-83af2867ebd3",
"46e07357-6946-4ff0-ba36-ae11840bdc39",
"026dac2d-2ab3-4f9c-9e1d-6198db4fb080",
"2f46d1e6-2a85-4d46-b955-10c2eded661c",
"14bcb17c-f70a-41d5-b10d-294388084dfc",
"487d752c-6e22-43e3-9c08-627bc2a6c6d4",
"543e4c99-5d9f-407d-be75-c397d633fe56",
"24ba7993-7fc8-4447-afd5-7ff6d548b11a",
"e09a667f-04bc-49b5-8246-daf248a29174",
"b17ff269-ec2d-4ad8-88aa-b00b75921427",
"58b2a4a4-b721-4753-bfc3-478cdb5bd1a8",
"28e0affc-cacb-4db8-ab32-dfc16931b86a",
"e72082e8-f9e6-42ac-ac45-de30f9adee9d",
"f0204f83-ea03-4c33-b7e7-13d2188ab3e5",
"9fabfbc8-1d5c-495e-9bb2-03795f0145ae",
"01ce4fd6-197a-4792-8778-775b03780369",
"47bb1fd4-d41f-42b4-8d0c-29c4e9fdff9f",
"7e8d03f2-2ff9-431d-af81-e5ffcd954a63",
"89857b33-fa50-469a-bbb3-91c8ab655931",
"5c2e910c-84e0-4042-b5d6-880a731c3e67",
"737e9619-7768-407c-8a4f-6fe1e8d61f04",
"abab0e69-f7e4-40c1-aa58-375798df487a"
]
import copy
df_rag_g = copy.deepcopy(df_rag)
df_norag_g = copy.deepcopy(df_norag)
# df_1direct_g = copy.deepcopy(df_1direct)
# df_1goalmediation_g = copy.deepcopy(df_1goalmediation)



df_rag_gold = df_rag_g[df_rag_g['source_uid'].isin(gold_list)]
df_norag_gold = df_norag_g[df_norag_g["source_uid"].isin(gold_list)]
# df_1direct_gold = df_1direct_g[df_1direct_g["source_uid"].isin(gold_list)]
# df_1goalmediation_gold = df_1goalmediation_g[df_1goalmediation_g["source_uid"].isin(gold_list)]

# SILVER (0)
silver_list = [
"02a06bf1-51b8-4902-b088-573e29fcd7ec",
"1dc85adb-fbdd-4275-b9cf-42976acb4d14",
"2978ddbc-cdc9-4bfa-9a7c-4bf056904010",
"2bc7d6fa-a02e-4367-b316-d6b4e8a2ce3f",
"31d6fe77-da70-42da-8f47-66bb79b9285b",
"321b5e21-2951-40c9-a2f9-6ce0c145cfb8",
"341b5211-bb72-4bec-bd3d-c0d518887960",
"35080724-6604-401c-8b06-19b7cece3d45",
"38a7b760-56f9-4565-8b70-f8dad5768ace",
"5461912b-69cd-40d7-8f79-50832f92f049",
"6dafeac7-75b6-4d69-96f7-d08708a0a99e",
"98434f4c-6216-4067-ad59-4a89cb47bb9b",
"a267b011-b1db-4e3c-aa49-438e2afdd6dc",
"b83285c5-0b88-4ced-a52e-5c34ea371507",
"debfb68a-eae2-464e-847a-cd3fea23f3ca",
"e4ad6fd7-2e3e-4991-b392-a0056f702286",
"e6231d1a-1f7f-4198-a499-7635509adfaf",
"ec3556de-be79-4ad4-aa0f-eaca48abb5d5",
"ed60dcdb-b273-44e7-b5dc-f9527d7c403f",
"f5ac654b-8f39-427b-856f-4a9a2d4a3020",
"fea524d4-a1b6-466c-ac48-8777c3fd173d"
]
df_rag_s = copy.deepcopy(df_rag)
df_norag_s = copy.deepcopy(df_norag)
# df_1direct_s = copy.deepcopy(df_1direct)
# df_1goalmediation_s = copy.deepcopy(df_1goalmediation)

df_rag_silver = df_rag_s[df_rag_s["source_uid"].isin(silver_list)]
df_norag_silver = df_norag_s[df_norag_s["source_uid"].isin(silver_list)]
# df_1direct_silver = df_1direct_s[df_1direct_s["source_uid"].isin(silver_list)]
# df_1goalmediation_silver = df_1goalmediation_s[df_1goalmediation_s["source_uid"].isin(silver_list)]
# BRONZE
bronze_list =[
"1a894d3c-b3ef-448a-a3de-2b38677cef36",
"29e00040-6e0f-4f0e-816d-1ac97c1e5485",
"2ba0becb-58c2-43a1-97bb-7e153a34eb47",
"2c27b5f1-4af6-49ad-a43c-3efb0c150868",
"2da5c1ee-bd40-406d-83a7-2f3d93293949",
"3728f856-0d47-4614-824f-37b6dda8e357",
"3ec3eab7-842d-409d-8866-42ddcbd24cd9",
"4fa75795-ddc4-4582-9715-bb7887439263",
"56fe0c73-77c4-40d9-a687-b2df28d5f7d7",
"5c15607b-96af-4503-84b4-d1745f3a3ae0",
"6628a2fb-19e2-4fe5-aedb-92fe5ceee9c9",
"690f58f1-f18c-4415-bab0-787c2f83d051",
"6ac1d2ed-1f6b-4828-a1ab-f81c40bd5e80",
"748536e4-636a-4dc6-b1a7-d9cbfdc1cffd",
"892629b0-61eb-425d-97f4-7d213074c435",
"907fd0e7-6821-4e2d-9c62-6d7afad5a9d1",
"a6419de9-1e40-4793-b21b-9c8d9038835a",
"ab7ed4f7-10ee-4ccb-bb21-4853c9018b1e",
"ae2d99c2-1720-4354-bc4d-f7bc3e4ee28d",
"b4072935-56a6-4765-bb4d-d5f6bbeb95b9",
"cf95d6a4-6ad7-462c-9700-9f04bd993667",
"d7a2e92e-dc74-4e79-be04-a86f829fc3ec",
"daf5384b-ea5c-4cce-bb8a-540a360075bf",
"e250017c-16ff-4825-9c30-160f391e1549",
"edc1869c-8a97-44fd-ab47-63fda4a54df9",
"grp-690f58f1-f18c-4415-bab0-787c2f83d051",
"grp-b59f7f5d-2991-49a6-8e88-0e2f2db92585",
"grp-ffd863cb-f06b-404e-a013-54acb61f1ed9"
]
df_rag_b = copy.deepcopy(df_rag)
df_norag_b = copy.deepcopy(df_norag)
# df_1direct_b = copy.deepcopy(df_1direct)
# df_1goalmediation_b = copy.deepcopy(df_1goalmediation)

df_rag_bronze = df_rag_b[df_rag_b["source_uid"].isin(bronze_list)]
df_norag_bronze = df_norag_b[df_norag_b["source_uid"].isin(bronze_list)]
# df_1direct_bronze = df_1direct_b[df_1direct_b["source_uid"].isin(bronze_list)]
# df_1goalmediation_bronze = df_1goalmediation_b[df_1goalmediation_b["source_uid"].isin(bronze_list)]

print(df_rag.shape[0])

print(df_rag_gold.shape[0])
print(df_norag_gold.shape[0])
# print(df_1direct_gold.shape[0])
# print(df_1goalmediation_gold.shape[0])

print(df_rag_silver.shape[0])
print(df_rag_bronze.shape[0])

# print(df_1direct_gold.shape[0])
# print(df_1direct_silver.shape[0])
# print(df_1direct_bronze.shape[0])

568
176
176
176
176
168
224
176
168
224


In [22]:
# %matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt




import plotly.express as px
def plot_sequence_cursor_interactive(df: pd.DataFrame, mybool: bool, key2plot: str, bin_num:int):
    df = df.dropna(subset=['inclusion_ratio', 'sequence_similarity_sbert', 'sequence_boolean'])
    filtered_df = df[df['sequence_boolean'] == mybool]
    size1 = df.shape[0]
    size2 = filtered_df.shape[0]
    print(size1, size2)

    #===============================
    #ratio of selection
    #===============================
    # Step 1: Bin the data by inclusion_ratio
    bin_width = 1.0/bin_num
    # bins = np.arange(0, 1.0001 + bin_width, bin_width)
    bins = np.arange(0, 0.9001 + bin_width, bin_width)
    df['inclusion_bin'] = pd.cut(df['inclusion_ratio'], bins=bins, include_lowest=True)

    # Step 2: Group by bins and calculate ratio
    grouped = df.groupby('inclusion_bin')
    bin_centers = [interval.mid for interval in grouped.groups.keys()]

    ratios = []
    for bin_label, group in grouped:
        total = len(group)
        true_count = group[group['sequence_boolean'] == True].shape[0]
        ratio = true_count / total if total > 0 else 0
        ratios.append(ratio)

    #===============================
    # Polynomial fit
    N = 1
    coeffs = np.polyfit(filtered_df['inclusion_ratio'], filtered_df[key2plot], deg=N)
    poly_func = np.poly1d(coeffs)
    x_smooth = np.linspace(filtered_df['inclusion_ratio'].min(), filtered_df['inclusion_ratio'].max(), 200)
    y_smooth = poly_func(x_smooth)

    # Scatter plot with hover info
    fig = px.scatter(
        filtered_df,
        x='inclusion_ratio',
        y=key2plot,
        hover_data=['idx','source_uid'],
        labels={
            'inclusion_ratio': 'Inclusion Ratio',
            key2plot: key2plot,
        },
        title=f'Inclusion Ratio vs. {key2plot} (bool={mybool}) {size2}/{size1}',
    )

    #===============================
    # Add polynomial fit line
    fig.add_scatter(
        x=x_smooth,
        y=y_smooth,
        mode='lines',
        name=f'{N}° polynomial fit',
        line=dict(color='red')
    )

    fig.update_layout(
        xaxis=dict(range=[-0.1, 1.1]),
        yaxis=dict(range=[0, 1.1]),
    )

    #===============================
    #ratio of selection
    #===============================
    # Add ratio line trace with secondary y-axis
    fig.add_trace(go.Scatter(
        x=bin_centers,
        y=ratios,
        mode='lines+markers',
        name='Ratio (mybool=True / all)',
        line=dict(color='green', dash='dot'),
        yaxis='y2'
    ))

    # Configure secondary y-axis
    fig.update_layout(
        yaxis2=dict(
            title='Ratio',
            overlaying='y',
            side='right',
            range=[0, 1.1]
        )
    )

    fig.show()

import plotly.graph_objects as go
def plot_taxonomy_cursor_interactive(df: pd.DataFrame, togglebool: bool, mybool: bool, key2plot: str, bin_num:int):
    # Clean data
    df = df.dropna(subset=['inclusion_ratio', 'taxonomy_similarity_sbert', 'taxonomy_boolean', 'taxonomy_weighted_similarity'])
    size1 = df.shape[0]
    
    # Filter based on toggle
    if togglebool:
        filtered_df = df[df['taxonomy_boolean'] == mybool]
    else:
        filtered_df = df
    size2 = filtered_df.shape[0]

    #===============================
    #ratio of selection
    #===============================
    # Step 1: Bin the data by inclusion_ratio
    bin_width = 1.0/bin_num
    # bins = np.arange(0, 1.0001 + bin_width, bin_width)
    bins = np.arange(0, 0.9001 + bin_width, bin_width)
    df['inclusion_bin'] = pd.cut(df['inclusion_ratio'], bins=bins, include_lowest=True)

    # Step 2: Group by bins and calculate ratio
    grouped = df.groupby('inclusion_bin')
    bin_centers = [interval.mid for interval in grouped.groups.keys()]

    ratios = []
    for bin_label, group in grouped:
        total = len(group)
        true_count = group[group['taxonomy_boolean'] == True].shape[0]
        ratio = true_count / total if total > 0 else 0
        ratios.append(ratio)


    # Polynomial fit (linear)
    N = 1
    coeffs = np.polyfit(filtered_df['inclusion_ratio'], filtered_df[key2plot], deg=N)
    poly_func = np.poly1d(coeffs)
    x_smooth = np.linspace(filtered_df['inclusion_ratio'].min(), filtered_df['inclusion_ratio'].max(), 200)
    y_smooth = poly_func(x_smooth)

    # Plotly scatter plot
    fig = px.scatter(
        filtered_df,
        x='inclusion_ratio',
        y=key2plot,
        hover_data=['source_uid', 'idx'],
        labels={
            'inclusion_ratio': 'Inclusion Ratio',
            key2plot: 'Taxonomy Similarity'
        },
        title=f'Inclusion Ratio vs. {key2plot} (taxonomy_boolean={mybool}) {size2}/{size1}'
    )

    # Add polynomial fit line
    fig.add_scatter(
        x=x_smooth,
        y=y_smooth,
        mode='lines',
        name=f'{N}° polynomial fit',
        line=dict(color='red')
    )

    # Adjust axis and layout
    fig.update_layout(
        xaxis=dict(range=[-0.1, 1.1]),
        yaxis=dict(range=[0, 1.1]),
        height=600,
        width=800
    )

    #===============================
    #ratio of selection
    #===============================
    # Add ratio line trace with secondary y-axis
    fig.add_trace(go.Scatter(
        x=bin_centers,
        y=ratios,
        mode='lines+markers',
        name='Ratio (mybool=True / all)',
        line=dict(color='green', dash='dot'),
        yaxis='y2'
    ))

    # Configure secondary y-axis
    fig.update_layout(
        yaxis2=dict(
            title='Ratio',
            overlaying='y',
            side='right',
            range=[0, 1.1]
        )
    )

    fig.show()


#taxonomy_similarity_sbert
#taxonomy_weighted_similarity

plot_taxonomy_cursor_interactive(df_rag, False, True, 'taxonomy_similarity_sbert', 9)
plot_taxonomy_cursor_interactive(df_norag, False, True, 'taxonomy_similarity_sbert',9)

plot_taxonomy_cursor_interactive(df_rag, False, True, 'taxonomy_weighted_similarity', 9)
plot_taxonomy_cursor_interactive(df_norag, False, True, 'taxonomy_weighted_similarity',9)
# plot_taxonomy_cursor(df_norag)

# plot_taxonomy_cursor(df_rag_gold, True)
# plot_taxonomy_cursor(df_rag_gold, False)
# plot_taxonomy_cursor(df_norag_gold, True)
# plot_taxonomy_cursor(df_norag_gold, False)

# plot_taxonomy_cursor(df_rag_silver, True)
# plot_taxonomy_cursor(df_rag_silver, False)
# plot_taxonomy_cursor(df_norag_silver, True)
# plot_taxonomy_cursor(df_norag_silver, False)

# plot_taxonomy_cursor(df_rag_bronze, True)
# plot_taxonomy_cursor(df_rag_bronze, False)
# plot_taxonomy_cursor(df_norag_bronze, True)
# plot_taxonomy_cursor(df_norag_bronze, False)



plot_sequence_cursor_interactive(df_rag, True, 'sequence_similarity_sbert', 9)
plot_sequence_cursor_interactive(df_norag, True, 'sequence_similarity_sbert',9)
# plot_sequence_cursor(df_1direct, True, 'sequence_similarity_sbert')
# plot_sequence_cursor_interactive(df_1goalmediation, True, 'sequence_similarity_sbert')
plot_sequence_cursor_interactive(df_raggoal, True, 'sequence_similarity_sbert', 9)
plot_sequence_cursor_interactive(df_noraggoal, True, 'sequence_similarity_sbert', 9)


plot_sequence_cursor_interactive(df_rag, True, 'sequence_dtw_similarity', 9)
plot_sequence_cursor_interactive(df_norag, True, 'sequence_dtw_similarity',9)
# plot_sequence_cursor(df_1direct, True, 'sequence_dtw_similarity')
# plot_sequence_cursor_interactive(df_1goalmediation, True, 'sequence_dtw_similarity')
plot_sequence_cursor_interactive(df_raggoal, True, 'sequence_dtw_similarity', 9)
plot_sequence_cursor_interactive(df_noraggoal, True, 'sequence_dtw_similarity', 9)

# plot_sequence_cursor(df_rag_gold, True)
# plot_sequence_cursor(df_rag_gold, False)
# plot_sequence_cursor(df_norag_gold, True)
# plot_sequence_cursor(df_norag_gold, False)
# plot_sequence_cursor(df_1goalmediation_gold, True)
# plot_sequence_cursor(df_1goalmediation_gold, False)

# plot_sequence_cursor(df_rag_silver, True)
# plot_sequence_cursor(df_rag_silver, False)
# plot_sequence_cursor(df_norag_silver, True)
# plot_sequence_cursor(df_1goalmediation_silver, True)
# plot_sequence_cursor(df_1goalmediation_silver, False)


# plot_sequence_cursor(df_rag_bronze, True)
# plot_sequence_cursor(df_rag_bronze, False)
# plot_sequence_cursor(df_norag_bronze, True)
# plot_sequence_cursor(df_norag_bronze, False)
# plot_sequence_cursor(df_1goalmediation_bronze, True)
# plot_sequence_cursor(df_1goalmediation_bronze, False)





/tmp/ipykernel_2107141/1259517961.py:120: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/tmp/ipykernel_2107141/1259517961.py:120: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/tmp/ipykernel_2107141/1259517961.py:120: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/tmp/ipykernel_2107141/1259517961.py:120: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



213 104


...

/tmp/ipykernel_2107141/1259517961.py:26: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/tmp/ipykernel_2107141/1259517961.py:26: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



213 117


213 110


/tmp/ipykernel_2107141/1259517961.py:26: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mplcursors

# Dummy DataFrame
df = pd.DataFrame({
    'inclusion_ratio': np.random.rand(10),
    'sequence_similarity_sbert': np.random.rand(10),
    'taxonomy_boolean': [True] * 10,
    'source_uid': [f"Item {i}" for i in range(40, 50)]
})

# plot_sequence_cursor(df)

## v6_evaluator Plots

evaluator_1: (core)  
evaluator_2: (entity)(core)  
evaluator_3: (entity)(sim)(core)  
evaluator_h: (entity)(sim)(human)  

In [62]:
path_eval_results = constants_init.PATH_ROOT + "f4_evaluate/result_v6/"
baseline_folders = ['output-rag-0602','output-norag-0602','output-1goalmediation-0602','output-1goalmediation-norag-0602']

def read_dicts2list(path):
    dict_list = []
    file_paths = os.listdir(path)
    for i in range(len(file_paths)):
        file_path = path + f"/evaluator_{i}.pkl"
        with open(file_path, 'rb') as f:
            dict = pickle.load(f)
            dict_list.append(dict)
    return dict_list

def convert_dtypes(df):
    if df.shape[1] == 13:  # goal baseline
        conversion_strings = ['entity_check_seq', 'core_check_seq', 'human_check_tax', 'human_check_seq']
    elif df.shape[1] == 19:  # ours baseline
        conversion_strings = ['entity_check_tax', 'entity_check_seq', 'core_check_tax', 'core_check_seq', 'human_check_tax', 'human_check_seq']
    else:
        print("convert_dtypes: THIS FUNC NEEDS ADJUSTMENT")
        return df

    for col in conversion_strings:
        if col in df.columns:
            df[col] = df[col].map(lambda x: True if str(x).lower() == 'true' or x == 1 else False)
        else:
            print(f"Warning: column '{col}' not found in DataFrame")

    return df
        
dict_eval_rag = read_dicts2list(path_eval_results + baseline_folders[0])
dict_eval_norag = read_dicts2list(path_eval_results + baseline_folders[1])
dict_eval_raggoal = read_dicts2list(path_eval_results + baseline_folders[2])
dict_eval_noraggoal = read_dicts2list(path_eval_results + baseline_folders[3])

# evaluator dictionaries
df_eval_rag = pd.DataFrame(dict_eval_rag)
df_eval_norag = pd.DataFrame(dict_eval_norag)
df_eval_raggoal = pd.DataFrame(dict_eval_raggoal)
df_eval_noraggoal = pd.DataFrame(dict_eval_noraggoal)

# bool are stored as object type. convert this to bool
df_eval_rag = convert_dtypes(df_eval_rag)
df_eval_norag = convert_dtypes(df_eval_norag)
df_eval_raggoal = convert_dtypes(df_eval_raggoal)
df_eval_noraggoal = convert_dtypes(df_eval_noraggoal)

print(df_eval_norag['core_check_seq'].dtype)


bool


In [ ]:
#evaluator_rag_:

#core check has 3 versions, tax, seq, both
efil1_tax_rag = df_eval_rag[df_eval_rag['core_check_tax']==True]
efil1_seq_rag = df_eval_rag[df_eval_rag['core_check_seq']==True]
efil1_both_rag = df_eval_rag[(df_eval_rag['core_check_tax']==True)&(df_eval_rag['core_check_seq']==True)]
print(f"rag core check (tax, seq, both): {efil1_tax_rag.shape[0]} {efil1_seq_rag.shape[0]} {efil1_both_rag.shape[0]}")

#entity check should both pass for (tax, seq) because otherwise sequence generation is impossible
efil2_tax_rag = df_eval_rag[(df_eval_rag['entity_check_tax']==True)&(df_eval_rag['entity_check_seq']==True)&(df_eval_rag['core_check_tax']==True)]
efil2_seq_rag = df_eval_rag[(df_eval_rag['entity_check_tax']==True)&(df_eval_rag['entity_check_seq']==True)&(df_eval_rag['core_check_seq']==True)]
efil2_both_rag = df_eval_rag[(df_eval_rag['entity_check_tax']==True)&(df_eval_rag['entity_check_seq']==True)&(df_eval_rag['core_check_tax']==True)&(df_eval_rag['core_check_seq']==True)]
print(f"rag entity check core(tax, seq, both): {efil2_tax_rag.shape[0]} {efil2_seq_rag.shape[0]} {efil2_both_rag.shape[0]}")

#similarity threshold should both be applied (tax, seq) because otherwise sequence generation is impossible
sim_tax_threshold = 0.5 #main itself is 50% weight
sim_seq_threshold = 0.3 #same threhold as goalmediation

efil3_tax_rag = df_eval_rag[(df_eval_rag['entity_check_tax']==True)&(df_eval_rag['entity_check_seq']==True)&(df_eval_rag['sim_tax3']>sim_tax_threshold)&(df_eval_rag['sim_seq3']>sim_seq_threshold)&(df_eval_rag['core_check_tax']==True)]
efil3_seq_rag = df_eval_rag[(df_eval_rag['entity_check_tax']==True)&(df_eval_rag['entity_check_seq']==True)&(df_eval_rag['sim_tax3']>sim_tax_threshold)&(df_eval_rag['sim_seq3']>sim_seq_threshold)&(df_eval_rag['core_check_seq']==True)]
efil3_both_rag = df_eval_rag[(df_eval_rag['entity_check_tax']==True)&(df_eval_rag['entity_check_seq']==True)&(df_eval_rag['sim_tax3']>sim_tax_threshold)&(df_eval_rag['sim_seq3']>sim_seq_threshold)&(df_eval_rag['core_check_tax']==True)&(df_eval_rag['core_check_seq']==True)]
print(f"rag entity/sim check core(tax, seq, both): {efil3_tax_rag.shape[0]} {efil3_seq_rag.shape[0]} {efil3_both_rag.shape[0]}")

rag core check (tax, seq, both): 140 119 107
rag entity check core(tax, seq, both): 115 97 93
rag entity/sim check core(tax, seq, both): 104 88 86


In [104]:
#evaluator_norag_: entity_check_tax, entity_check_seq, sim_tax1, sim_tax2, sim_tax3, sim_seq1, sim_seq2, sim_seq3, core_check_tax, core_check_tax
#core check has 3 versions, tax, seq, both
efil1_tax_norag = df_eval_norag[df_eval_norag['core_check_tax']==True]
efil1_seq_norag = df_eval_norag[df_eval_norag['core_check_seq']==True]
efil1_both_norag = df_eval_norag[(df_eval_norag['core_check_tax']==True)&(df_eval_norag['core_check_seq']==True)]
print(f"rag core check (tax, seq, both): {efil1_tax_norag.shape[0]} {efil1_seq_norag.shape[0]} {efil1_both_norag.shape[0]}")

#entity check should both pass for (tax, seq) because otherwise sequence generation is impossible
efil2_tax_norag = df_eval_norag[(df_eval_norag['entity_check_tax']==True)&(df_eval_norag['entity_check_seq']==True)&(df_eval_norag['core_check_tax']==True)]
efil2_seq_norag = df_eval_norag[(df_eval_norag['entity_check_tax']==True)&(df_eval_norag['entity_check_seq']==True)&(df_eval_norag['core_check_seq']==True)]
efil2_both_norag = df_eval_norag[(df_eval_norag['entity_check_tax']==True)&(df_eval_norag['entity_check_seq']==True)&(df_eval_norag['core_check_tax']==True)&(df_eval_norag['core_check_seq']==True)]
print(f"rag entity check core(tax, seq, both): {efil2_tax_norag.shape[0]} {efil2_seq_norag.shape[0]} {efil2_both_norag.shape[0]}")

#similarity threshold should both be applied (tax, seq) because otherwise sequence generation is impossible
sim_tax_threshold = 0.5 #main itself is 50% weight
sim_seq_threshold = 0.3 #same threhold as goalmediation

efil3_tax_norag = df_eval_norag[(df_eval_norag['entity_check_tax']==True)&(df_eval_norag['entity_check_seq']==True)&(df_eval_norag['sim_tax3']>sim_tax_threshold)&(df_eval_norag['sim_seq3']>sim_seq_threshold)&(df_eval_norag['core_check_tax']==True)]
efil3_seq_norag = df_eval_norag[(df_eval_norag['entity_check_tax']==True)&(df_eval_norag['entity_check_seq']==True)&(df_eval_norag['sim_tax3']>sim_tax_threshold)&(df_eval_norag['sim_seq3']>sim_seq_threshold)&(df_eval_norag['core_check_seq']==True)]
efil3_both_norag = df_eval_norag[(df_eval_norag['entity_check_tax']==True)&(df_eval_norag['entity_check_seq']==True)&(df_eval_norag['sim_tax3']>sim_tax_threshold)&(df_eval_norag['sim_seq3']>sim_seq_threshold)&(df_eval_norag['core_check_tax']==True)&(df_eval_norag['core_check_seq']==True)]
print(f"rag entity/sim check core(tax, seq, both): {efil3_tax_norag.shape[0]} {efil3_seq_norag.shape[0]} {efil3_both_norag.shape[0]}")

rag core check (tax, seq, both): 139 127 112
rag entity check core(tax, seq, both): 107 108 95
rag entity/sim check core(tax, seq, both): 100 102 89


In [ ]:
#evaluaotr_raggoal_ entity_check_seq, sim_seq1, sim_seq2, sim_seq3", core_check_seq, human_check_tax, human_check_seq
sim_seq_threshold = 0.35
efil1_raggoal = df_eval_raggoal[df_eval_raggoal['core_check_seq']==True]
efil2_raggoal = df_eval_raggoal[(df_eval_raggoal['entity_check_seq']==True) & (df_eval_raggoal['core_check_seq']==True)]
efil3_raggoal= df_eval_raggoal[(df_eval_raggoal['entity_check_seq']==True) & (df_eval_raggoal['sim_seq3']>sim_seq_threshold) & (df_eval_raggoal['core_check_seq']==True)]
print(f"rag (core) (entity,core) (entity,sim,core): {efil1_raggoal.shape[0]} {efil2_raggoal.shape[0]} {efil3_raggoal.shape[0]}")
# efilh_raggoal

#evaluaotr_noraggoal_ entity_check_seq, sim_seq1, sim_seq2, sim_seq3", core_check_seq, human_check_tax, human_check_seq
efil1_noraggoal = df_eval_noraggoal[df_eval_noraggoal['core_check_seq']==True]
efil2_noraggoal = df_eval_noraggoal[(df_eval_noraggoal['entity_check_seq']==True) & (df_eval_noraggoal['core_check_seq']==True)]
efil3_noraggoal = df_eval_noraggoal[(df_eval_noraggoal['entity_check_seq']==True) & (df_eval_noraggoal['sim_seq3']> sim_seq_threshold) & (df_eval_noraggoal['core_check_seq'])]
print(f"norag (core) (entity,core) (entity,sim,core): {efil1_noraggoal.shape[0]} {efil2_noraggoal.shape[0]} {efil3_noraggoal.shape[0]}")

rag (core) (entity,core) (entity,sim,core): 132 106 104
norag (core) (entity,core) (entity,sim,core): 130 95 93


In [ ]:
#HUMAN EVALUATOR가 가장 이상적이라고 가정할 때에 다른 evaluator에 대해서 precision-recall curve를 그릴 수 있다.
#PRECISION = TP / (TP+FP)예측한 positive
#RECALL = TP / (TP+FN)실제 positive



# PLOT
#-------------------------------------
# 베이스라인이 similarity의 변화에 따라서 어느 정도의 수용성 그래프를 보이는지 그린다.(false/positive + positive/positive)
# input (baselines, filtered_baselines, titlename)
# x = inclusion_ratio
# y = acceptance_rate per bin
#-------------------------------------
